In [1]:
import requests
from tvDatafeed import TvDatafeed,Interval
import pandas as pd
import talib
from sklearn.impute import SimpleImputer
from datetime import datetime, timedelta
imputer = SimpleImputer(strategy='mean')


In [2]:
path = pd.read_csv('NIFTY.csv')
five_min_data= pd.DataFrame(path)

five_min_data['datetime']=pd.to_datetime(five_min_data['datetime'])
five_min_data['year'] = five_min_data['datetime'].dt.year
five_min_data['hour'] = five_min_data['datetime'].dt.hour
five_min_data['minute'] = five_min_data['datetime'].dt.minute

#candlestick Patterns, macd, ema_14_period

five_min_data['EMA_14'] = talib.EMA(five_min_data['close'],timeperiod=14)
macd, macdsignal, macdhist = talib.MACD(five_min_data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
five_min_data['MACD'] = macd
five_min_data['engulfing'] = talib.CDLENGULFING(five_min_data['open'], five_min_data['high'], five_min_data['low'],five_min_data['close'])
#five_min_data['dragonfly'] = talib.CDLDRAGONFLYDOJI(five_min_data['open'], five_min_data['high'], five_min_data['low'],five_min_data['close'])
five_min_data['hammer'] = talib.CDLHAMMER(five_min_data['open'], five_min_data['high'], five_min_data['low'],five_min_data['close'])
#five_min_data['gravestone_doji'] = talib.CDLGRAVESTONEDOJI(five_min_data['open'], five_min_data['high'], five_min_data['low'],five_min_data['close'])
five_min_data['shooting_star'] = talib.CDLSHOOTINGSTAR(five_min_data['open'], five_min_data['high'], five_min_data['low'],five_min_data['close'])
print(five_min_data)
five_min_data.columns

                datetime     symbol      open      high       low     close  \
0    2023-08-31 12:25:00  NSE:NIFTY  19299.75  19305.90  19292.05  19305.50   
1    2023-08-31 12:30:00  NSE:NIFTY  19305.75  19308.40  19301.40  19302.00   
2    2023-08-31 12:35:00  NSE:NIFTY  19303.10  19304.60  19297.90  19300.70   
3    2023-08-31 12:40:00  NSE:NIFTY  19302.50  19311.85  19297.05  19308.95   
4    2023-08-31 12:45:00  NSE:NIFTY  19309.65  19317.95  19302.55  19308.05   
...                  ...        ...       ...       ...       ...       ...   
4995 2023-12-08 15:05:00  NSE:NIFTY  20960.25  20973.05  20957.55  20972.40   
4996 2023-12-08 15:10:00  NSE:NIFTY  20973.40  20979.15  20964.25  20978.15   
4997 2023-12-08 15:15:00  NSE:NIFTY  20977.90  20978.20  20967.15  20974.80   
4998 2023-12-08 15:20:00  NSE:NIFTY  20975.55  20979.55  20966.80  20977.90   
4999 2023-12-08 15:25:00  NSE:NIFTY  20978.20  20983.00  20970.65  20972.10   

         volume  year  hour  minute        EMA_14  

Index(['datetime', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'year',
       'hour', 'minute', 'EMA_14', 'MACD', 'engulfing', 'hammer',
       'shooting_star'],
      dtype='object')

In [3]:
five_min_data=five_min_data.iloc[201:].reset_index(drop=True)
start_ind=[]
X_train=[]
y_train=[]
y_train_pos=[]
gain=[]

for j in range(len(five_min_data.index)):
    if five_min_data['hour'][j]==9 and five_min_data['minute'][j]==15: # Getting the indices where the trading day starts
        start_ind.append(j)
end_ind=[] # end of the day indices
for j in range(1,len(start_ind)): #starting from 1 because we dont want an end without the start of a day which is the case for start_of_day[0]-1
    end_ind.append(start_ind[j]-1)


if len(start_ind)>len(end_ind):
    start_ind=start_ind[:-1]

j=0
while j<len(start_ind):
    if end_ind[j]-start_ind[j]!=74:
        end_ind.remove(end_ind[j])
        start_ind.remove(start_ind[j])
        continue
    df_new=five_min_data.iloc[start_ind[j]+2:end_ind[j]-2].drop(['symbol','datetime','year','hour','minute'],axis=1)

    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------#
    df_new = pd.DataFrame(imputer.fit_transform(df_new))
    if len(df_new.columns)==len(five_min_data.columns)-5:
        X_train.append(df_new.values.flatten())
    else:
        end_ind.remove(end_ind[j])
        start_ind.remove(start_ind[j])
    j+=1

for j in end_ind:
    gain_inst=(five_min_data['close'][j+7]-five_min_data['close'][j-2])#*100/five_min_data['close'][j-2] # gain at the particular instant diff. between 10:00am - 11:30am
    gain.append(gain_inst)  # i+2 indicates the next day at 9:20(as we need to let the price settle) and i-2 at time 3:15 as we need to trade before the day ends and we cant trade ecaxtly at the end of the day
    
y_train=gain

In [4]:
from sklearn.preprocessing import MinMaxScaler

# Normalizing the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import shuffle

#shuffled data
X_train_shuffled, y_train_shuffled = shuffle(X_train, y_train, random_state=42)

rf_regressor=RandomForestRegressor(n_estimators=60,random_state=42)

rf_regressor.fit(X_train_shuffled,y_train_shuffled)

RandomForestRegressor(n_estimators=60, random_state=42)

In [6]:
tv=TvDatafeed()

df = tv.get_hist(symbol='NIFTY',exchange='NSE',interval=Interval.in_5_minute,n_bars=800)
df_n= pd.DataFrame(df)
df_n=df_n.reset_index()
df_n['datetime']=pd.to_datetime(df_n['datetime'])
df_n['year'] = df_n['datetime'].dt.year
df_n['hour'] = df_n['datetime'].dt.hour
df_n['minute'] = df_n['datetime'].dt.minute
#----------------------------------------------------------------------------------------------------------------------------------------------------------#

df_n['EMA_14'] = talib.EMA(df_n['close'],timeperiod=14)
macd, macdsignal, macdhist = talib.MACD(df_n['close'], fastperiod=12, slowperiod=26, signalperiod=9)
df_n['MACD'] = macd
df_n['engulfing'] = talib.CDLENGULFING(df_n['open'], df_n['high'], df_n['low'],df_n['close'])
#df_n['dragonfly'] = talib.CDLDRAGONFLYDOJI(df_n['open'], df_n['high'], df_n['low'],df_n['close'])
df_n['hammer'] = talib.CDLHAMMER(df_n['open'], df_n['high'], df_n['low'],df_n['close'])
#df_n['gravestone_doji'] = talib.CDLGRAVESTONEDOJI(df_n['open'], df_n['high'], df_n['low'],df_n['close'])
df_n['shooting_star'] = talib.CDLSHOOTINGSTAR(df_n['open'], df_n['high'], df_n['low'],df_n['close'])
#df_n.to_csv('test1.csv')
df_n= df_n.iloc[201:].reset_index(drop=True)
#df_n.to_csv('test2.csv')
#--------------------------------------------------------------------------------------------------------------------------------------------------------#
engulf=[]
engulf_sell=[0]

for i in df_n.index:                    
    if df_n['engulfing'][i] >0 :
        engulf.append(i)
  

y = len(df_n.index)-engulf[-1]

x_pred=[]
price_arr=[]

df_n=df_n.reset_index(drop=True)
df_nn=df_n.iloc[-69-y:-y+1].drop(['symbol','datetime','year','hour','minute'],axis=1)
#df_nn.to_csv('trading data.csv')
x_pred.append(df_nn.values.flatten())
x_pred_norm=scaler.transform(x_pred)
y_pred=rf_regressor.predict(x_pred)
z=df_n['datetime'][engulf[-1]]
print(f"\nPrdecting Nifty to go {y_pred} points from {z} NIFTY at {df_n['close'][engulf[-1]]}")
print(f"\nEntry time :- {z}\nStop Loss at: {.3*y_pred} \nProfit book at : {.9*y_pred}")


##------------------------------------------------------------------------------------------------------------------------------------------------------##

url = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'

headers= {"Accept-Encoding" : "gzip, deflate, br",
          "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
          "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

session = requests.Session()
data = session.get(url, headers=headers).json()['records']['data']
ocdata =[]
for i in data:
    for j,k in i.items():
        if j=='CE' or j=="PE":
            info=k
            info["instrumentType"]= j
            ocdata.append(info)

ddf = pd.DataFrame(ocdata).drop(['underlying', 'openInterest','changeinOpenInterest', 'pchangeinOpenInterest', 'totalTradedVolume','impliedVolatility', 'change', 'pChange','totalBuyQuantity', 'totalSellQuantity', 'bidQty', 'bidprice', 'askQty','askPrice', 'underlyingValue', 'instrumentType'],axis=1)

subt = (df_n['close'][engulf[-1]])%50
st_price=(int(df_n['close'][engulf[-1]]-subt))
options = []
for i in ddf.index:
    if ddf['strikePrice'][i] == st_price:
        options.append(i)

intiatl_bal = 25000
call_position=[]
sell_position=[]
for i in options[0:2]:
    if y_pred>0 and "CE" in ddf['identifier'][i]:
        call_posn = int(intiatl_bal/ddf['lastPrice'][i])
        call_posn = int(call_posn/50)
        print(f"\nBuy {call_posn} lot of CE strike price: -- {ddf['strikePrice'][i]}")

    elif y_pred<0 and "PE" in ddf['identifier'][i]:
        call_posn = int(intiatl_bal/ddf['lastPrice'][i])
        call_posn = int(call_posn/50)
        print(f"\n Buy {call_posn} lot of PE strike price: -- {ddf['strikePrice'][i]}")
    



you are using nologin method, data you access may be limited



Prdecting Nifty to go [104.8725] points from 2024-01-15 15:15:00 NIFTY at 22100.5

Entry time :- 2024-01-15 15:15:00
Stop Loss at: [31.46175] 
Profit book at : [94.38525]

Buy 4 lot of CE strike price: -- 22100
